<a href="https://colab.research.google.com/github/beaten-by-the-market/krxnewsscrap/blob/main/%EC%98%A4%EB%8A%98%EC%9D%98_%EC%BD%94%EC%8A%A4%ED%94%BC_%EB%B2%88%EC%97%AD%EB%8C%80%EC%83%81%EA%B3%B5%EC%8B%9C.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install mysql-connector-python
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime, timedelta
import time
import mysql.connector

# 전달받은 DB 인자를 입력
print("db_user를 입력하세요")
db_user = input()
print("db_password를 입력하세요")
db_password = input()
print("db_host를 입력하세요")
db_host = input()
# 조회할 날짜를 YYYY-MM-DD 형식으로 입력
print('조회할 날짜를 YYYY-MM-DD 형식으로 입력하세요')
today_date = input()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.0/34.0 MB 46.8 MB/s eta 0:00:00


In [15]:
#-----------------------------------------------------------
# 환경변수 설정
#-----------------------------------------------------------
db_config = {
    'user': db_user,
    'password': db_password,
    'host': db_host,
    'port': 3306,
    'database': 'englishkind',
}

# 데이터사용
def load_data_from_db():
    connection = None
    try:
        # 데이터베이스 연결
        connection = mysql.connector.connect(**db_config)

        if connection.is_connected():
            print("Connected to MySQL database")

            # 커서 생성
            cursor = connection.cursor()

            # 데이터 가져오기
            query = "SELECT * FROM kospi_report"
            cursor.execute(query)

            # 데이터프레임 초기화
            disc = pd.DataFrame(columns=[desc[0] for desc in cursor.description])

            # 배치로 데이터를 가져오는 로직 (이 부분은 그대로 유지)
            batch_size = 1000  # 한 번에 가져올 행의 수
            rows_fetched = 0

            while True:
                rows = cursor.fetchmany(batch_size)
                if not rows:
                    break
                disc = pd.concat([disc, pd.DataFrame(rows, columns=disc.columns)], ignore_index=True)

                rows_fetched += len(rows)

            return disc

    except mysql.connector.Error as e:
        print(f"Error: {e}")
        return pd.DataFrame()  # 빈 데이터프레임 반환

    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

# SQL 데이터를 최초 1회만 로드
# 지원대상 데이터 리스트 (df_data 정의)
df_data = load_data_from_db()

# 컬럼명 설정
df_data.columns = ["서식코드", "서식명", "대분류", "구분", "업데이트일"]

# DataFrame 설정
df_svc = df_data

# 데이터사용
def load_data_from_db_name():
    connection = None
    try:
        # 데이터베이스 연결
        connection = mysql.connector.connect(**db_config)

        if connection.is_connected():
            print("Connected to MySQL database")

            # 커서 생성
            cursor = connection.cursor()

            # 데이터 가져오기
            query = "SELECT * FROM kospi_companies"
            cursor.execute(query)

            # 데이터프레임 초기화
            disc = pd.DataFrame(columns=[desc[0] for desc in cursor.description])

            # 배치로 데이터를 가져오는 로직 (이 부분은 그대로 유지)
            batch_size = 1000  # 한 번에 가져올 행의 수
            rows_fetched = 0

            while True:
                rows = cursor.fetchmany(batch_size)
                if not rows:
                    break
                disc = pd.concat([disc, pd.DataFrame(rows, columns=disc.columns)], ignore_index=True)

                rows_fetched += len(rows)

            return disc

    except mysql.connector.Error as e:
        print(f"Error: {e}")
        return pd.DataFrame()  # 빈 데이터프레임 반환

    finally:
        if connection.is_connected():
            cursor.close()
            connection.close()
            print("MySQL connection is closed")

# SQL 데이터를 최초 1회만 로드
# 지원대상 데이터 리스트 (df_data 정의)
df_data = load_data_from_db_name()

# 컬럼명 설정
df_data.columns = ["회사코드", "회사명", "상장여부", "업데이트일"]

# DataFrame 생성
df_listed = df_data

# 모든 페이지의 데이터를 저장할 빈 리스트
all_data = []

# 페이지별 데이터 수집 함수
def get_page_data(page_num):
    url = 'https://kind.krx.co.kr/disclosure/todaydisclosure.do'
    params = {
        "method": "searchTodayDisclosureSub",
        "currentPageSize": 100,
        "pageIndex": page_num,
        "orderMode": 0,
        "orderStat": "D",
        "marketType": 1,
        "forward": "todaydisclosure_sub",
        "searchMode": "",
        "searchCodeType": "",
        "chose": "S",
        "todayFlag": "Y",
        "repIsuSrtCd": "",
        "kosdaqSegment": "",
        "selDate": today_date,
        "searchCorpName": "",
        "copyUrl": ""
    }

    try:
        response = requests.post(url, params=params)
        response.raise_for_status()
        return BeautifulSoup(response.text, 'html.parser')
    except Exception as e:
        print(f"페이지 {page_num} 요청 중 오류 발생: {e}")
        return None

# 데이터 파싱 함수
def parse_table(soup):
    data = []
    table = soup.find('table', class_='list type-00 mt10')

    if table and table.find('tbody'):
        for row in table.find('tbody').find_all('tr'):
            cols = row.find_all('td')

            if len(cols) >= 5:
                # 필요한 데이터 추출 (원본 코드와 동일)
                time = cols[0].text.strip()

                company_a_tag = cols[1].find('a', id='companysum')
                company = company_a_tag.text.strip() if company_a_tag else ""

                company_code = ""
                if company_a_tag and company_a_tag.has_attr('onclick'):
                    onclick_attr = company_a_tag['onclick']
                    code_match = re.search(r"companysummary_open\('([A-Za-z0-9]+)'\)", onclick_attr)
                    if code_match:
                        company_code = code_match.group(1)

                title_a_tag = cols[2].find('a')
                title = ""
                note = ""

                if title_a_tag:
                    title = title_a_tag.get('title', "").strip()

                    font_tags = title_a_tag.find_all('font')
                    if font_tags:
                        notes = []
                        for font_tag in font_tags:
                            notes.append(font_tag.text.strip())
                        note = "_".join(notes)

                submitter = cols[3].text.strip()

                discl_url = ""
                if title_a_tag and title_a_tag.has_attr('onclick'):
                    onclick_attr = title_a_tag['onclick']
                    match = re.search(r"openDisclsViewer\('(\d+)'", onclick_attr)
                    if match:
                        acptno = match.group(1)
                        discl_url = f"https://kind.krx.co.kr/common/disclsviewer.do?method=search&acptno={acptno}&docno=&viewerhost=&viewerport="

                data.append({
                    '시간': time,
                    '회사코드': company_code,
                    '회사명': company,
                    '비고': note,
                    '공시제목': title,
                    '제출인': submitter,
                    '상세URL': discl_url
                })
    return data

# 첫 페이지 요청 및 데이터 처리
url = 'https://kind.krx.co.kr/disclosure/todaydisclosure.do'
params = {
    "method": "searchTodayDisclosureSub",
    "currentPageSize": 100,
    "pageIndex": 1,
    "orderMode": 0,
    "orderStat": "D",
    "marketType": 1,
    "forward": "todaydisclosure_sub",
    "searchMode": "",
    "searchCodeType": "",
    "chose": "S",
    "todayFlag": "Y",
    "repIsuSrtCd": "",
    "kosdaqSegment": "",
    "selDate": today_date,
    "searchCorpName": "",
    "copyUrl": ""
}

response = requests.post(url, params=params)
soup = BeautifulSoup(response.text, 'html.parser')

# 총 건수와 페이지 수 추출
total_items_element = soup.select_one('.info.type-00 em')
total_pages_text = soup.select_one('.info.type-00').text.strip()
total_pages_match = re.search(r'(\d+)/(\d+)', total_pages_text)

if total_items_element and total_pages_match:
    total_items = int(total_items_element.text.strip().replace(",",""))
    total_pages = int(total_pages_match.group(2))

    print(f"조회일에 총 {total_items}건의 공시가 있습니다. (총 {total_pages}페이지)    지원대상 공시는 아래 표를 참고해주세요.")
else:
    print("페이지 정보를 찾을 수 없습니다.")
    total_pages = 1

# 첫 페이지 데이터 처리
all_data.extend(parse_table(soup))

# 페이지가 여러 개인 경우 나머지 페이지 처리
if total_pages > 1:


    for i, page in enumerate(range(2, total_pages + 1)):
        page_soup = get_page_data(page)
        if page_soup:
            page_data = parse_table(page_soup)
            all_data.extend(page_data)

        # 진행률 업데이트
        progress = (i + 1) / (total_pages - 1)


        # 서버 부하를 줄이기 위한 대기 시간
        time.sleep(0.5)

# 데이터프레임 생성
df_discl = pd.DataFrame(all_data)

# 필터링 (지원 대상 서식만 필터)
form_names = df_svc['서식명'].unique().tolist()

def is_contained(title):
    for form_name in form_names:
        if form_name in title:
            return True
    return False

# 첫 번째 필터링: 지원 대상 서식만 필터
filtered_df = df_discl[df_discl['공시제목'].apply(is_contained)]

# 두 번째 필터링: 지정된 회사 코드만 필터
listed_company_codes = df_listed['회사코드'].tolist()
filtered_df = filtered_df[filtered_df['회사코드'].isin(listed_company_codes)]

# 결과 표시

if filtered_df.empty:
    print("조건에 맞는 공시 데이터가 없습니다.")
else:
    print(f"총 {len(filtered_df)}건의 지원대상 공시가 있습니다.")

filtered_df

Connected to MySQL database
MySQL connection is closed
Connected to MySQL database
MySQL connection is closed
조회일에 총 30건의 공시가 있습니다. (총 1페이지)    지원대상 공시는 아래 표를 참고해주세요.
총 2건의 지원대상 공시가 있습니다.


,시간,회사코드,회사명,비고,공시제목,제출인,상세URL
3,11:10,31740,자이에스앤디,,단일판매ㆍ공급계약체결,자이에스앤디,https://kind.krx.co.kr/common/disclsviewer.do?...
23,09:15,33626,두산퓨얼셀,,단일판매ㆍ공급계약체결,두산퓨얼셀,https://kind.krx.co.kr/common/disclsviewer.do?...
